<a href="https://colab.research.google.com/github/rohitdavas/NLP/blob/master/Hugging_face_NLP_Chapter_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

notes from hugging face course : https://huggingface.co/learn/nlp-course/chapter3/2?fw=pt

In [1]:
!pip install datasets evaluate transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# taste of how single step of fintuning works. 

In [2]:
# creating a fine tuning model for sequence classificiation 

import torch 
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

In [3]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

print(model)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [4]:
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]

batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
batch["labels"] = torch.tensor([1, 1])

for key, value in batch.items():
    print(f"key = {key} \n\tshape of value = {value.shape}, \n\tvalue={value}")


key = input_ids 
	shape of value = torch.Size([2, 16]), 
	value=tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  2023,  2607,  2003,  6429,   999,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0]])
key = token_type_ids 
	shape of value = torch.Size([2, 16]), 
	value=tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
key = attention_mask 
	shape of value = torch.Size([2, 16]), 
	value=tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
key = labels 
	shape of value = torch.Size([2]), 
	value=tensor([1, 1])


In [5]:
# initlialise the optimizer
optimizer = AdamW(model.parameters()) # deprecated in transformers. warning.

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [6]:
optimizer.zero_grad()
loss = model(**batch).loss
print(loss)

loss.backward()
optimizer.step()

# run it over time and again to see the model overfitting. 

tensor(0.5613, grad_fn=<NllLossBackward0>)


# adding more data


- working with mrpc dataset. 
- contains the pair of sentences, and label if they resemble the same meaning.

In [7]:
from datasets import load_dataset
raw_datasets = load_dataset("glue", "mrpc")
print(raw_datasets)

  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})


In [8]:
raw_dataset_train = raw_datasets["train"]
for feature, info in raw_dataset_train.features.items():
    print(feature, " - ", info)

print("-----")
for feature, value in raw_dataset_train[0].items():
    print(f"feature = {feature}\n\tvalue = {value}")

sentence1  -  Value(dtype='string', id=None)
sentence2  -  Value(dtype='string', id=None)
label  -  ClassLabel(names=['not_equivalent', 'equivalent'], id=None)
idx  -  Value(dtype='int32', id=None)
-----
feature = sentence1
	value = Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .
feature = sentence2
	value = Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .
feature = label
	value = 1
feature = idx
	value = 0


```python

tokenized_dataset = tokenizer(
    raw_datasets["train"]["sentence1"],
    raw_datasets["train"]["sentence2"],
    padding=True,
    truncation=True,
)
```

This works well, but it has the disadvantage of returning a dictionary (with our keys, input_ids, attention_mask, and token_type_ids, and values that are lists of lists). It will also only work if you have enough RAM to store your whole dataset during the tokenization (whereas the datasets from the 🤗 Datasets library are Apache Arrow files stored on the disk, so you only keep the samples you ask for loaded in memory).

In [9]:
# this is a function to define what to do with a single instance of example.

def tokenizer_map_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

# the tokenizer is backed by the internal function written in Rust. 
# the significant advantage is when we process a large amount of data on it.

tokenized_datasets = raw_datasets.map(tokenizer_map_function, batched=True)

# padding is more efficient when done on batch label. that saves us the cost of padding 
# till the maximum length of tokenizer support, 
# and use the maximum of the batch as proxy - called Dynamic Padding. 

print(tokenized_datasets)

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})


Dynamic Padding

In [10]:
# pad only by the need of the batch rather than a fixed amount always.
# pros : speeds up the training. 
# cons : TPUs still need the fixed padding. 

# the collate function is the one that aggregates the samples of a batch together.
# simply : concatenate recursively all the elements of an iteratable object.

# hugging face have an implementation of DataCollatorWithPadding
from transformers import DataCollatorWithPadding 
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# test it with a small sample.
samples  = tokenized_datasets["train"][:8]
samples = {k:v for k, v in samples.items() if k not in ["sentence1", "sentence2", "idx"]}
print("lengths of the elements : ", [len(x) for x in samples["input_ids"]])

# batch them with the DataCollatorWithPadding : a dynamic padding 
batch = data_collator(samples)
print(f"shapes of batch elements: ", {k:v.shape for k, v in batch.items()})

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


lengths of the elements :  [50, 59, 47, 67, 59, 50, 62, 32]
shapes of batch elements:  {'input_ids': torch.Size([8, 67]), 'token_type_ids': torch.Size([8, 67]), 'attention_mask': torch.Size([8, 67]), 'labels': torch.Size([8])}


# Summary from above and trainer API

In [11]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding 

# --------------------------
# Data part for MRPC dataset
# ---------------------------

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased" # checkpoint name 
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], 
                     example["sentence2"], 
                     truncation=True)
    
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # for dynamic padding. 




  0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

In [12]:
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
from transformers import TrainingArguments, Trainer, AutoModelForSequenceClassification 

In [17]:
import evaluate 
import numpy as np

def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds 
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [18]:
# check a number of arguments within 
training_args = TrainingArguments(output_dir="test-trainer", evaluation_strategy = "epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model=model,
    args=training_args, 
    train_dataset = tokenized_datasets["train"], 
    eval_dataset = tokenized_datasets["validation"],
    data_collator = data_collator, # also a default so no worries.
    tokenizer = tokenizer,
    compute_metrics=compute_metrics,
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [19]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.516200
1000,0.280400


TrainOutput(global_step=1377, training_loss=0.3177691108520774, metrics={'train_runtime': 225.5069, 'train_samples_per_second': 48.797, 'train_steps_per_second': 6.106, 'total_flos': 406183858377360.0, 'train_loss': 0.3177691108520774, 'epoch': 3.0})